# Bash

***
###  

In [4]:
if [ ! -d "./04_Motif_Comparison_Output" ]; then
  mkdir 04_Motif_Comparison_Outputr
fi

## 1. Generate meme file

In [2]:
seqkit tab2fx ./03_Nuc_Cyto_kmer_assembly_Output/nuc_favour_motif_consensus.txt | seqkit seq  --dna2rna > ./04_Motif_Comparison_Output/nuc_favour_motif_consensus_seq_rna.fa
rna2meme ./04_Motif_Comparison_Output/nuc_favour_motif_consensus_seq_rna.fa > ./04_Motif_Comparison_Output/nuc_favour_motif_consensus_seq_rna.meme

seqkit tab2fx ./03_Nuc_Cyto_kmer_assembly_Output/cyto_favour_motif_consensus.txt | seqkit seq  --dna2rna  > ./04_Motif_Comparison_Output/cyto_favour_motif_consensus_seq_rna.fa
rna2meme ./04_Motif_Comparison_Output/cyto_favour_motif_consensus_seq_rna.fa > ./04_Motif_Comparison_Output/cyto_favour_motif_consensus_seq_rna.meme

***
###  

## 2. Compare with known RNA motifs by Tomtom
- Motif Databases were downloaded from https://meme-suite.org/meme/meme-software/Databases/motifs/motif_databases.12.21.tgz

In [3]:
tomtom -no-ssc -oc . -verbosity 1 -min-overlap 5 -dist pearson -evalue -thresh 0.1 \
-norc ./04_Motif_Comparison_Output/nuc_favour_motif_consensus_seq_rna.meme ../../motif_databases/CISBP-RNA/Homo_sapiens.meme -o ./04_Motif_Comparison_Output/nuc_tomtom

tomtom -no-ssc -oc . -verbosity 1 -min-overlap 5 -dist pearson -evalue -thresh 0.1 \
-norc ./04_Motif_Comparison_Output/cyto_favour_motif_consensus_seq_rna.meme ../../motif_databases/CISBP-RNA/Homo_sapiens.meme -o ./04_Motif_Comparison_Output/cyto_tomtom

***
###  

## 3. Get related RBP names

In [5]:
# Remove redundant and keep the best match
grep "M*_0.6" ./04_Motif_Comparison_Output/nuc_tomtom/tomtom.tsv  | sort -k2,2 -k6,6n -u | awk '!a[$2]++'  > ./04_Motif_Comparison_Output/nuc_favour_motif_tomtom.tsv
grep "M*_0.6" ./04_Motif_Comparison_Output/cyto_tomtom/tomtom.tsv  | sort -k2,2 -k6,6n -u | awk '!a[$2]++'  > ./04_Motif_Comparison_Output/cyto_favour_motif_tomtom.tsv

RBP="../../motif_databases/CISBP-RNA/Homo_sapiens.meme"

# get Nuc_Related RBP name
touch Nuc_Related_RBP.temp
cat ./04_Motif_Comparison_Output/nuc_favour_motif_tomtom.tsv | grep "M*_0.6" | while read line 
do
    Target_ID=`echo "$line" | cut -f2`
    grep -w $Target_ID $RBP | cut -d" " -f3 >>  Nuc_Related_RBP.temp
done
paste Nuc_Related_RBP.temp ./04_Motif_Comparison_Output/nuc_favour_motif_tomtom.tsv > ./04_Motif_Comparison_Output/nuc_favour_motif_final.tsv #

# get Cyto_Related RBP name
cat ./04_Motif_Comparison_Output/cyto_favour_motif_tomtom.tsv | grep "M*_0.6"  | while read line 
do
    Target_ID=`echo "$line" | cut -f2`
    grep -w $Target_ID $RBP | cut -d" " -f3 >>  Cyto_Related_RBP.temp
done
paste Cyto_Related_RBP.temp ./04_Motif_Comparison_Output/cyto_favour_motif_tomtom.tsv> ./04_Motif_Comparison_Output/cyto_favour_motif_final.tsv #
rm Nuc_Related_RBP.temp Cyto_Related_RBP.temp